In [4]:
from dataload import get_dataloaders

train,test,val,info = get_dataloaders('DermaMNIST',batch_size=32)

Using downloaded and verified file: /home/lotfimecharbat/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/lotfimecharbat/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/lotfimecharbat/.medmnist/dermamnist.npz


In [ ]:
from ofa.imagenet_classification.networks import ResNets
from ofa.imagenet_classification.elastic_nn.modules.dynamic_layers import DynamicMBConvLayer, DynamicConvLayer, DynamicLinearLayer
import torch

super_net = torch.hub.load('mit-han-lab/once-for-all', 'ofa_supernet_resnet50', pretrained=True)
n_classes = 40

# Modify classifier for the dataset
super_net.classifier = DynamicLinearLayer([super_net.classifier.linear.linear.in_features], n_classes)



Using cache found in /home/lotfimecharbat/.cache/torch/hub/mit-han-lab_once-for-all_master
Downloading: "https://huggingface.co/han-cai/ofa/resolve/main/ofa_supernet_resnet50" to .torch/ofa_nets/ofa_supernet_resnet50
/home/lotfimecharbat/Projects/MedNNS/ofa/model_zoo.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any 

In [7]:
# test supernet
x = torch.randn(1, 3, 244, 244)
logits = super_net(x)
print(logits.shape)

torch.Size([1, 40])


In [31]:
import random
# test subnetwork
def sample_active_subnet(supernet, sample_config):
    """
    Sample and set an active subnet configuration.
    """
    # sample expand ratio
    if isinstance(sample_config['e'][0], list):
        expand_setting = [random.choice(sample_config['e'][i]) for i in range(len(sample_config['e']))]
    else:
        expand_setting = [random.choice(sample_config['e']) for _ in range(len(supernet.blocks))]

    # sample depth
    depth_setting = []
    for stage_id in range(len(sample_config['d'])):
        depth_list = sample_config['d'][stage_id]
        depth_setting.append(random.choice(depth_list))

    # sample width_mult
    width_mult_index = [random.choice(sample_config['w'][i]) for i in range(len(sample_config['w']))]
    width_mult_setting = [
        list(range(len(supernet.input_stem[0].out_channel_list)))[width_mult_index[0]],
        list(range(len(supernet.input_stem[2].out_channel_list)))[width_mult_index[1]],
    ]
    for stage_id, block_idx in enumerate(supernet.grouped_block_index):
        stage_first_block = supernet.blocks[block_idx[0]]
        width_mult_setting.append(
            list(range(len(stage_first_block.out_channel_list)))[width_mult_index[stage_id+2]]
        )

    arch_config = {"d": depth_setting, "e": expand_setting, "w": width_mult_setting}
    supernet.set_active_subnet(**arch_config)
    return arch_config

sample_config={"d": [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]],
        "e": [0.35],
        "w": [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]]}

arch_config = sample_active_subnet(super_net, sample_config)
subnet=super_net.get_active_subnet()
# calculate n params
print(arch_config)
sum(p.numel() for p in subnet.parameters())- sum(p.numel() for p in super_net.classifier.parameters())

{'d': [2, 1, 0, 2, 1], 'e': [0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35], 'w': [1, 0, 2, 2, 1, 1]}


24502520

In [6]:
import torch
from torch.utils.data import DataLoader
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import random
import torch.nn.functional as F
import os
import numpy as np

from medmnist import INFO, Evaluator
from medmnist import (DermaMNIST, PneumoniaMNIST, PathMNIST, RetinaMNIST, 
                      BreastMNIST, BloodMNIST, ChestMNIST, OCTMNIST, 
                      OrganCMNIST, OrganSMNIST, OrganAMNIST, TissueMNIST)
import torchvision.transforms as transforms
from ofa.imagenet_classification.networks import ResNets
from ofa.imagenet_classification.elastic_nn.modules.dynamic_layers import (
    DynamicMBConvLayer, DynamicConvLayer, DynamicLinearLayer
)

# Dataset mapping configuration
DATA_CLASS_MAPPING = {
    'PathMNIST': PathMNIST,
    'ChestMNIST': ChestMNIST,
    'DermaMNIST': DermaMNIST,
    'OCTMNIST': OCTMNIST,
    'PneumoniaMNIST': PneumoniaMNIST,
    'RetinaMNIST': RetinaMNIST,
    'BreastMNIST': BreastMNIST,
    'BloodMNIST': BloodMNIST,
    'TissueMNIST': TissueMNIST,
    'OrganAMNIST': OrganAMNIST,
    'OrganCMNIST': OrganCMNIST,
    'OrganSMNIST': OrganSMNIST
}

DATA_PATH_MAPPING = {}
Main_dir = './work_dir/Supernets'
for ds in DATA_CLASS_MAPPING.keys():
    dir_path = os.path.join(Main_dir, ds,"super_net_complete_with_sandwich.pth")
    DATA_PATH_MAPPING[ds] = dir_path

print(DATA_PATH_MAPPING)





def sample_active_subnet(supernet, sample_config):
    """
    Sample an active subnet configuration from the supernet.
    
    Args:
        supernet: The supernet model
        sample_config (dict): Configuration for sampling
        
    Returns:
        dict: Architecture configuration
    """
    # Sample expand ratio
    expand_setting = []
    for block in supernet.blocks:
        expand_setting.append(random.choice(sample_config['e']))
    
    # Sample depth
    depth_list = sample_config['d'][0]
    depth_setting = [random.choice(depth_list)]
    for stage_id in range(len(ResNets.BASE_DEPTH_LIST)):
        depth_list = sample_config['d'][stage_id + 1]
        depth_setting.append(random.choice(depth_list))
    
    # Sample width multiplier
    width_mult_index = [random.choice(sample_config['w'][i]) 
                       for i in range(len(sample_config['w']))]
    width_mult_setting = [
        list(range(len(supernet.input_stem[0].out_channel_list)))[width_mult_index[0]],
        list(range(len(supernet.input_stem[2].out_channel_list)))[width_mult_index[1]],
    ]
    for stage_id, block_idx in enumerate(supernet.grouped_block_index):
        stage_first_block = supernet.blocks[block_idx[0]]
        width_mult_setting.append(
            list(range(len(stage_first_block.out_channel_list)))[width_mult_index[stage_id + 2]]
        )
    
    arch_config = {"d": depth_setting, "e": expand_setting, "w": width_mult_setting}
    supernet.set_active_subnet(**arch_config)
    return arch_config


def validate(val_loader, model, criterion, device):
    """
    Validate model on given data loader.
    
    Args:
        val_loader: Data loader for validation
        model: Model to validate
        criterion: Loss function
        device: Device to run on
        
    Returns:
        tuple: (average_loss, accuracy)
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Handle label format
            if labels.size(1) > 1:
                labels = torch.argmax(labels, dim=1)
            else:
                labels = labels.squeeze()
            
            # Handle single channel images
            if images.size(1) == 1:
                images = images.repeat(1, 3, 1, 1)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
    
    avg_loss = running_loss / len(val_loader.dataset)
    accuracy = 100.0 * correct / total
    return avg_loss, accuracy


def sample_fixed_subnets(supernet, num_samples=10, seed=42):
    """
    Sample a fixed set of subnets using a deterministic seed.
    
    Args:
        supernet: The supernet model
        num_samples (int): Number of subnets to sample
        seed (int): Random seed for reproducibility
        
    Returns:
        list: List of sampled configurations
    """
    random.seed(seed)
    sampled_configs = []
    sample_config = {
        'd': [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]],
        'e': [0.2, 0.25, 0.35],
        'w': [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0,1,2]]
    }
    
    for _ in range(num_samples):
        sampled_configs.append(sample_active_subnet(supernet, sample_config))
    
    return sampled_configs


def evaluate_subnets(supernet, sampled_configs, val_loader, test_loader, device):
    """
    Evaluate sampled subnets and compute performance metrics.
    
    Args:
        supernet: The supernet model
        sampled_configs (list): List of subnet configurations
        val_loader: Validation data loader
        test_loader: Test data loader
        device: Device to run on
        
    Returns:
        list: List of evaluation results
    """
    results = []
    criterion = torch.nn.CrossEntropyLoss()
    supernet.to(device)
    supernet.eval()
    
    for config in tqdm(sampled_configs, desc="Evaluating sampled subnets"):
        supernet.set_active_subnet(**config)
        subnet = supernet.get_active_subnet()
        
        # Compute validation and test accuracy
        val_loss, val_acc = validate(val_loader, subnet, criterion, device)
        test_loss, test_acc = validate(test_loader, subnet, criterion, device)
        
        results.append({
            "subnet_config": {'w': config['w'], 'd': config['d'],'e': config['e']},
            "num_parameters": sum(p.numel() for p in subnet.parameters()),
            "val_accuracy": val_acc,
            "test_accuracy": test_acc,
        })
    
    return results


def get_functional_encoding(model, num_samples=64, seed=42):
    """
    Compute functional encoding of a model using embeddings of Gaussian noise.
    
    Args:
        model: Model to encode
        num_samples (int): Number of noise samples
        seed (int): Random seed
        
    Returns:
        torch.Tensor: Functional encoding
    """
    torch.manual_seed(seed)
    x = torch.randn(num_samples, 3, 224, 224)
    linear = model.classifier.copy()
    model.classifier = torch.nn.Identity()
    embed = model(x)
    model.classifier = linear
    return embed


def load_and_evaluate_supernet(dataset_name, device='cuda:0', num_samples=1000):
    """
    Load supernet and evaluate subnets for a given dataset.
    
    Args:
        dataset_name (str): Name of the dataset
        work_dir (str): Working directory path
        device (str): Device to use
        num_samples (int): Number of subnets to sample
        
    Returns:
        list: Evaluation results
    """
    
    model_path = DATA_PATH_MAPPING.get(dataset_name)
    
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Supernet not found at {model_path}")
    
    # Load supernet
    supernet = torch.hub.load('mit-han-lab/once-for-all', 'ofa_supernet_resnet50', pretrained=True)
    n_classes = 7  # Assuming 7 classes for DermaMNIST, adjust as needed
    
    # Modify classifier for the dataset
    supernet.classifier = DynamicLinearLayer([supernet.classifier.linear.linear.in_features], n_classes)
    supernet = supernet.to(device)
    supernet.load_state_dict(torch.load(model_path, map_location=device)['state_dict'])
    
    # Load data
    val_loader, test_loader = get_dataloaders(dataset_name=dataset_name,batch_size=256)
    
    # Sample and evaluate subnets
    sampled_configs = sample_fixed_subnets(supernet, num_samples=num_samples, seed=42)
    results = evaluate_subnets(supernet, sampled_configs, val_loader, test_loader, device)
    
    return results


def aggregate_dataset_results():
    """
    Aggregate results from multiple datasets into a single file.
    """
    # File containing dataset encodings
    encoding_file = './outputs/medmnist_resnet50_encodings.pth'
    
    # Load the dataset encodings
    dataset_encodings = torch.load(encoding_file)
    aggregated_data = []
    
    # Iterate over each dataset
    for dataset_name, encoding in dataset_encodings.items():
        if dataset_name not in DATA_PATH_MAPPING.keys():
            print(f"Warning: Dataset '{dataset_name}' not found in mapping. Skipping.")
            continue
            
        subnets_file = DATA_PATH_MAPPING.get(dataset_name)
        if not os.path.exists(subnets_file):
            print(f"Warning: Subnets file for '{dataset_name}' not found at {subnets_file}. Skipping.")
            continue
        
        # Load the subnets info
        subnets_info = torch.load(subnets_file)
        
        # Aggregate data for each subnet
        data_encoding = torch.mean(encoding, dim=0)
        
        for subnet_info in subnets_info:
            model_encoding = torch.mean(subnet_info.get("functional"), dim=0)
            
            sample = {
                "dataset_name": dataset_name,
                "dataset_encoding": data_encoding,
                "topology": subnet_info.get("topology"),
                "val_accuracy": subnet_info.get("val_accuracy"),
                "val_loss": subnet_info.get("val_loss"),
                "test_loss": subnet_info.get("test_loss"),
                "test_accuracy": subnet_info.get("test_accuracy"),
                "functional": model_encoding,
            }
            aggregated_data.append(sample)
    
    # Save aggregated data
    output_file = "aggregated_dataset_source.pth"
    torch.save(aggregated_data, output_file)
    print(f"Aggregated dataset source file saved to: {output_file}")


if __name__ == "__main__":
    # Example usage
    dataset_name = "DermaMNIST"
    results = load_and_evaluate_supernet(dataset_name, num_samples=10)

    print(f"Evaluation completed for {dataset_name}")
    print(f"Number of results: {len(results)}")
    print(results)
    
    # Aggregate results
    #aggregate_dataset_results()

{'PathMNIST': './work_dir/Supernets/PathMNIST/super_net_complete_with_sandwich.pth', 'ChestMNIST': './work_dir/Supernets/ChestMNIST/super_net_complete_with_sandwich.pth', 'DermaMNIST': './work_dir/Supernets/DermaMNIST/super_net_complete_with_sandwich.pth', 'OCTMNIST': './work_dir/Supernets/OCTMNIST/super_net_complete_with_sandwich.pth', 'PneumoniaMNIST': './work_dir/Supernets/PneumoniaMNIST/super_net_complete_with_sandwich.pth', 'RetinaMNIST': './work_dir/Supernets/RetinaMNIST/super_net_complete_with_sandwich.pth', 'BreastMNIST': './work_dir/Supernets/BreastMNIST/super_net_complete_with_sandwich.pth', 'BloodMNIST': './work_dir/Supernets/BloodMNIST/super_net_complete_with_sandwich.pth', 'TissueMNIST': './work_dir/Supernets/TissueMNIST/super_net_complete_with_sandwich.pth', 'OrganAMNIST': './work_dir/Supernets/OrganAMNIST/super_net_complete_with_sandwich.pth', 'OrganCMNIST': './work_dir/Supernets/OrganCMNIST/super_net_complete_with_sandwich.pth', 'OrganSMNIST': './work_dir/Supernets/Orga

Using cache found in /home/lotfimecharbat/.cache/torch/hub/mit-han-lab_once-for-all_master
/home/lotfimecharbat/Projects/MedNNS/ofa/model_zoo.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this e

Using downloaded and verified file: /home/lotfimecharbat/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/lotfimecharbat/.medmnist/dermamnist.npz


Evaluating sampled subnets: 100%|██████████| 10/10 [00:58<00:00,  5.83s/it]

Evaluation completed for DermaMNIST
Number of results: 10
[{'subnet_config': {'w': [0, 2, 0, 2, 2, 2], 'd': [0, 0, 0, 2, 2], 'e': [0.35, 0.2, 0.2, 0.35, 0.25, 0.2, 0.2, 0.2, 0.35, 0.2, 0.35, 0.35, 0.35, 0.2, 0.35, 0.25, 0.2, 0.2]}, 'num_parameters': 30036207, 'val_accuracy': 71.78464606181456, 'test_accuracy': 72.56857855361596}, {'subnet_config': {'w': [0, 2, 1, 2, 0, 1], 'd': [0, 1, 1, 2, 1], 'e': [0.35, 0.25, 0.2, 0.25, 0.35, 0.25, 0.2, 0.2, 0.35, 0.25, 0.25, 0.25, 0.2, 0.2, 0.25, 0.2, 0.2, 0.25]}, 'num_parameters': 12059463, 'val_accuracy': 71.5852442671984, 'test_accuracy': 71.52119700748129}, {'subnet_config': {'w': [1, 1, 0, 2, 1, 2], 'd': [1, 1, 2, 1, 0], 'e': [0.2, 0.35, 0.25, 0.35, 0.35, 0.25, 0.35, 0.2, 0.35, 0.2, 0.2, 0.35, 0.2, 0.25, 0.2, 0.2, 0.2, 0.25]}, 'num_parameters': 13334335, 'val_accuracy': 72.08374875373879, 'test_accuracy': 71.52119700748129}, {'subnet_config': {'w': [1, 1, 0, 0, 2, 2], 'd': [1, 0, 0, 0, 1], 'e': [0.35, 0.35, 0.2, 0.35, 0.35, 0.2, 0.35, 0.35, 0.

In [1]:
from meta_space_builder import model_encoder_main
results = model_encoder_main('DermaMNIST', device='cuda:0', num_samples=100)

{'PathMNIST': './work_dir/Supernets/PathMNIST/super_net_complete_with_sandwich.pth', 'ChestMNIST': './work_dir/Supernets/ChestMNIST/super_net_complete_with_sandwich.pth', 'DermaMNIST': './work_dir/Supernets/DermaMNIST/super_net_complete_with_sandwich.pth', 'OCTMNIST': './work_dir/Supernets/OCTMNIST/super_net_complete_with_sandwich.pth', 'PneumoniaMNIST': './work_dir/Supernets/PneumoniaMNIST/super_net_complete_with_sandwich.pth', 'RetinaMNIST': './work_dir/Supernets/RetinaMNIST/super_net_complete_with_sandwich.pth', 'BreastMNIST': './work_dir/Supernets/BreastMNIST/super_net_complete_with_sandwich.pth', 'BloodMNIST': './work_dir/Supernets/BloodMNIST/super_net_complete_with_sandwich.pth', 'TissueMNIST': './work_dir/Supernets/TissueMNIST/super_net_complete_with_sandwich.pth', 'OrganAMNIST': './work_dir/Supernets/OrganAMNIST/super_net_complete_with_sandwich.pth', 'OrganCMNIST': './work_dir/Supernets/OrganCMNIST/super_net_complete_with_sandwich.pth', 'OrganSMNIST': './work_dir/Supernets/Orga

Using cache found in /home/lotfimecharbat/.cache/torch/hub/mit-han-lab_once-for-all_master
/home/lotfimecharbat/Projects/MedNNS/ofa/model_zoo.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this e

Using downloaded and verified file: /home/lotfimecharbat/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/lotfimecharbat/.medmnist/dermamnist.npz


Evaluating sampled subnets: 100%|██████████| 100/100 [10:02<00:00,  6.02s/it]

Results saved to ./work_dir/Supernets/DermaMNIST/DermaMNIST_subnet_evaluation_results.pth
